# Advent of Code, 6th Dec 2024

## Guard Gallivant - Part 1

The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single **guard** is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
```

The map shows the current position of the guard with ^ (to indicate the guard is currently facing **up** from the perspective of the map). Any **obstructions** - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

If there is something directly in front of you, turn right 90 degrees.
Otherwise, take a step forward.
Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

```
....#.....
....^....#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:

```
....#.....
........>#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Reaching another obstacle (a spool of several **very** long polymers), she turns right again and continues downward:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#......v.
........#.
#.........
......#...
```

This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#v..
```

By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. **Including the guard's starting position**, the positions visited by the guard before leaving the area are marked with an X:

```
....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
```

In this example, the guard will visit **41** distinct positions on your map.

Predict the path of the guard. **How many distinct positions will the guard visit before leaving the mapped area?**

### Read in the data

In [2]:
#!F#
let inputFilePath = "aoc-2024-12-06-puzzle-input.txt"

Read the lines from the file.

In [20]:
open System.IO

let lines: string list = File.ReadAllLines(inputFilePath) |> Array.toList
lines.Length

130

In [21]:
lines |> List.map (fun str -> str.Length) |> Set.ofList

Count,1
(values),[ 130 ]


We see that the lines define a 130x130 square.

### Solve the problem

We need to convert the lines into a set of obstacle positions, and also the starting position of the guard.  We will mark the lower-left-hand corner as (0,0), so we will need to reverse the lines so that we start processing the lines from row 0.

In [22]:
let N = lines.Length
N

130

In [23]:
let identity n = n

let numberedLines = Seq.zip (Seq.initInfinite identity) (lines |> List.rev) |> Seq.toList
numberedLines[0]

Item1,0
Item2,..........#........#................#.........#...........................................#....#..#...........#..####..........#..


In [24]:
let obstaclePostionsSeq = seq {
    for row, line in numberedLines do
        let numberedChars: (int*char) list = Seq.zip (Seq.initInfinite identity) line |> Seq.toList
        for col, char in numberedChars do
            if (char = '#') then yield (col, row) // x, y
}
let obstaclePositions = obstaclePostionsSeq |> Set.ofSeq
obstaclePositions.Count

841

Now we get the guard's starting position.

In [25]:
let guardPostionsSeq = seq {
    for row, line in numberedLines do
        let numberedChars: (int*char) list = Seq.zip (Seq.initInfinite identity) line |> Seq.toList
        for col, char in numberedChars do
            if (char = '^') then yield (col,row)
}
let guardPostions = guardPostionsSeq |> Set.ofSeq
guardPostions.Count

1

As expected, there is only one guard position.

In [26]:
let guardPosition = guardPostions |> Set.toList |> List.item 0
guardPosition

Item1,91
Item2,76


In [27]:
type Direction = | Up | Right | Down | Left
type Guard = {position: (int*int); direction: Direction}

let nextDirection (guard: Guard): Direction = // guards always turn to the right
    match guard.direction with
    | Up -> Right
    | Right -> Down
    | Down -> Left
    | Left -> Up

let nextPosition (guard: Guard): int*int =
    let col, row = guard.position
    match guard.direction with
    | Up -> (col, row+1)
    | Right -> (col+1, row)
    | Down -> (col, row-1)
    | Left -> (col-1, row)

In [28]:
let guard = {position = guardPosition; direction = Up}

Now we iterate the guards movements until the guard leaves the grid, and track the guards positions.

In [29]:
let inBounds (guard: Guard): bool =
    let row, col = guard.position
    (row >= 0) && (row < N) && (col >= 0) && (col < N)

let rec iterateGuardPath (guard: Guard) (path: (int*int) Set): (int*int) Set =
    if (inBounds guard)
    then
        let newPosition = nextPosition guard
        if (obstaclePositions |> Set.contains newPosition)
        then
            let newDirection = nextDirection guard
            iterateGuardPath {guard with direction = newDirection} path
        else
            let newGuard = {guard with position = newPosition}
            if (inBounds newGuard)
            then iterateGuardPath newGuard (path |> Set.add newPosition)
            else path
    else path

Now we calculate the path (set of all positions) of the guard.

In [30]:
let path = iterateGuardPath guard ([guard.position] |> Set.ofList)
path.Count

5208

## Guard Gallivant - Part 2

While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and record the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are **pretty sure** that adding a single new obstruction **won't** cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get **stuck in a loop**, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know **all** of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only **6** different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use O to mark the new obstruction, | to show a position where the guard moves up/down, - to show a position where the guard moves left/right, and + to show a position where the guard moves both up/down and left/right.

Option one, put a printing press next to the guard's starting position:

```
....#.....
....+---+#
....|...|.
..#.|...|.
....|..#|.
....|...|.
.#.O^---+.
........#.
#.........
......#...
```

Option two, put a stack of failed suit prototypes in the bottom right quadrant of the mapped area:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
......O.#.
#.........
......#...
```

Option three, put a crate of chimney-squeeze prototype fabric next to the standing desk in the bottom right quadrant:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----+O#.
#+----+...
......#...
```

Option four, put an alchemical retroencabulator near the bottom left corner:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
..|...|.#.
#O+---+...
......#...
```

Option five, put the alchemical retroencabulator a bit to the right instead:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
....|.|.#.
#..O+-+...
......#...
```

Option six, put a tank of sovereign glue right next to the tank of universal solvent:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----++#.
#+----++..
......#O..
```

It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are **6** different positions you could choose.

You need to get the guard stuck in a loop by adding a single new obstruction. **How many different positions could you choose for this obstruction?**

### Solve the problem

First we need to find the unobstructed positions where we could put a new obstacle - so not an existing obstacle position, and not the guard's initial position.

In [31]:
let unobstructedPositionSeq = seq {
    for col in 0..(N-1) do
        for row in 0..(N-1) do
            let newPosition = (col,row)
            if (not (obstaclePositions |> Set.contains newPosition) && (guard.position <> newPosition))
            then yield newPosition
}
let unobstructedPositions = unobstructedPositionSeq |> Seq.toList
unobstructedPositions.Length

16058

In [32]:
obstaclePositions.Count + unobstructedPositions.Length + 1 = N*N

True

This time, when we track the guards path, we will need to capture the guard's direction and position both, as when we see the same position & direction a second time (without going out of bounds, the guard is in a loop).

In [33]:
type LoopResult = | Exit | Loop

let rec iterateGuardPath2 (obstacles: (int*int) Set) (guard: Guard) (path: Guard Set): LoopResult =
    if (inBounds guard)
    then
        let newPosition = nextPosition guard
        if (obstacles |> Set.contains newPosition)
        then
            let newDirection = nextDirection guard
            iterateGuardPath2 obstacles {guard with direction = newDirection} path
        else
            let newGuard = {guard with position = newPosition}
            if (path |> Set.contains newGuard)
            then Loop
            elif (inBounds newGuard)
            then iterateGuardPath2 obstacles newGuard (path |> Set.add newGuard)
            else Exit
    else Exit

let rec findLoopPositions (guard: Guard): (int*int) Set =
    let loopPositionsSeq = seq {
        for pos in unobstructedPositions do
            let newObstacles = obstaclePositions |> Set.add pos
            let newResult = iterateGuardPath2 newObstacles guard ([guard] |> Set.ofList)
            if (newResult = Loop)
            then yield pos
    }
    loopPositionsSeq |> Set.ofSeq

In [34]:
let loopPositions = findLoopPositions guard
loopPositions.Count

1972